In [42]:
import pandas as pd
import jpy_tools.parseSnake2 as jps

In [43]:
configPath = '/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml'
snakemakeFilePath = '/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/snakefile'

In [44]:
snakeFile = jps.SnakeFile()

In [45]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.addCode("sampleLs = list(config['Samples'].keys())")
config = snakeHeader.getConfig()
snakeHeader

/public/home/liuzj/softwares/anaconda3/lib/python3.8/site-packages/jpy_tools/parseSnake2.py:93: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.yaml = yaml.load(open(self.path))


import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())

In [46]:
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())

In [47]:
cellRangerRule = jps.SnakeRule(snakeFile, 'cellRanger', 48)
cellRangerRule.addCode("""
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
""")
cellRangerRule.addMetaDf('cellRangerParamsDf', ['outputDir', 'bamFile', 'barcodeFile', 'tenXResultDir'])
cellRangerRule.addMain('input', ['fastq'], )
cellRangerRule.addMain('params', ['includeIntron', 'expectCell', 'r2Length', 'id', 'outputDir','cellRangerIndex'])
cellRangerRule.setShell("""
mkdir -p {params.outputDir}
cd {params.outputDir}
if [ {params.includeIntron} = True ]
then
    cellranger6 count --id {params.id} --include-introns --transcriptome {params.cellRangerIndex} --fastqs {input.fastq} --expect-cells {params.expectCell} --localcores {threads} --r2-length {params.r2Length}
else
    cellranger6 count --id {params.id} --transcriptome {params.cellRangerIndex} --fastqs {input.fastq} --expect-cells {params.expectCell} --localcores {threads} --r2-length {params.r2Length}
fi
""")
cellRangerRule

2021-06-01 20:42:21.808 | INFO     | jpy_tools.parseSnake2:addRule:55 - cellRanger step num: 1


cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
for column in ['outputDir', 'bamFile', 'barcodeFile', 'tenXResultDir']:
    cellRangerParamsDf[column] = resultDir + 'step1_cellRanger/' + cellRangerParamsDf[column]
----------------
IN RULE
----------------
rule cellRanger:
    input:
        fastq = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'fastq'],
    output:
        cellRangerFinished = resultDir + 'step1_cellRanger/' + '{sample}.finished',
    params:
        gpu = 0,
       

In [48]:
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
cellRangerParamsDf

,fastq,includeIntron,expectCell,r2Length,id,outputDir,bamFile,barcodeFile,tenXResultDir,cellRangerIndex
artis_A3,/public/home/liuzj/projects/singleCell/singleC...,True,5000,91,artis_A3,artis_A3,artis_A3/artis_A3/outs/possorted_genome_bam.bam,artis_A3/artis_A3/outs/filtered_feature_bc_mat...,artis_A3/artis_A3,/public/home/liuzj/data/singleCellRef/mm10_UCS...
artis_A4,/public/home/liuzj/projects/singleCell/singleC...,True,5000,91,artis_A4,artis_A4,artis_A4/artis_A4/outs/possorted_genome_bam.bam,artis_A4/artis_A4/outs/filtered_feature_bc_mat...,artis_A4/artis_A4,/public/home/liuzj/data/singleCellRef/mm10_UCS...


In [49]:
parseUmiDrRule = jps.SnakeRule(snakeFile, 'parseUmiDr', 12)
parseUmiDrRule.addCode("""
parseUmiDrParamsDf = pd.DataFrame(index=sampleLs)
parseUmiDrParamsDf = parseUmiDrParamsDf.assign(output = lambda df:df.index + '_umiDr.tsv', bed = config['genomeBed'])
""")
parseUmiDrRule.addMetaDf('parseUmiDrParamsDf', ['output'])
parseUmiDrRule.addMain('params', ['bamFile', 'barcodeFile'], fromRule=cellRangerRule)
parseUmiDrRule.addMain('params', ['output', 'bed'])
parseUmiDrRule.setShell("""

python ~/scripts/tools/singleCell/parseUmiDirectionFromCellrangerBam.py \
    -i {params.bamFile} -o {params.output} -t {threads} --bed {params.bed} --barcode {params.barcodeFile}
""")
parseUmiDrRule

2021-06-01 20:42:22.534 | INFO     | jpy_tools.parseSnake2:addRule:55 - parseUmiDr step num: 2


parseUmiDrParamsDf = pd.DataFrame(index=sampleLs)
parseUmiDrParamsDf = parseUmiDrParamsDf.assign(output = lambda df:df.index + '_umiDr.tsv', bed = config['genomeBed'])
for column in ['output']:
    parseUmiDrParamsDf[column] = resultDir + 'step2_parseUmiDr/' + parseUmiDrParamsDf[column]
----------------
IN RULE
----------------
rule parseUmiDr:
    input:
        cellRangerFinished = resultDir + 'step1_cellRanger/' + '{sample}.finished',
    output:
        parseUmiDrFinished = resultDir + 'step2_parseUmiDr/' + '{sample}.finished',
    params:
        gpu = 0,
        bamFile = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'bamFile'],
        barcodeFile = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'barcodeFile'],
        output = lambda wildcard: parseUmiDrParamsDf.at[wildcard.sample, 'output'],
        bed = lambda wildcard: parseUmiDrParamsDf.at[wildcard.sample, 'bed'],
    threads:12
    shell:
        """
python ~/scripts/tools/singleCell/parseUmiDirectionFr

In [50]:
velocytoRule = jps.SnakeRule(snakeFile, 'velocyto', 12)
velocytoRule.addCode("""
velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf = velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')
""")
velocytoRule.addMetaDf('velocytoParamsDf')
velocytoRule.addMain('params', ['tenXResultDir'], fromRule=cellRangerRule)
velocytoRule.addMain('params', ['gtf'])
velocytoRule.setShell("""
velocyto run10x -@ {threads} --samtools-memory 2048 {params.tenXResultDir} {params.gtf}
""")
velocytoRule

2021-06-01 20:42:22.651 | INFO     | jpy_tools.parseSnake2:addRule:55 - velocyto step num: 3


velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf = velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')
----------------
IN RULE
----------------
rule velocyto:
    input:
        cellRangerFinished = resultDir + 'step1_cellRanger/' + '{sample}.finished',
    output:
        velocytoFinished = resultDir + 'step3_velocyto/' + '{sample}.finished',
    params:
        gpu = 0,
        tenXResultDir = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'tenXResultDir'],
        gtf = lambda wildcard: velocytoParamsDf.at[wildcard.sample, 'gtf'],
    threads:12
    shell:
        """
velocyto run10x -@ {threads} --samtools-memory 2048 {params.tenXResultDir} {params.gtf}
touch {output.velocytoFinished}
        """

In [51]:
velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')

,gtf
artis_A3,/public/home/liuzj/data/singleCellRef/mm10_UCS...
artis_A4,/public/home/liuzj/data/singleCellRef/mm10_UCS...


In [52]:
snakeAll = jps.SnakeAll(snakeFile, parseUmiDrRule, velocytoRule)
snakeAll

rule all:
    input:
        parseUmiDrFinished = [resultDir + 'step2_parseUmiDr/' + "" + sample + ".finished" for sample in sampleLs],
        velocytoFinished = [resultDir + 'step3_velocyto/' + "" + sample + ".finished" for sample in sampleLs],

In [53]:
snakeFile.getMain(snakemakeFilePath)

import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
for column in ['outputDir', 'bamFile', 'barcodeFile', 'tenXResultDir']:
    cellRangerParamsDf[column] = resultDir + 'step1_cellRanger/' + cellRangerParamsDf[colum